In [1]:
import nest_asyncio
import discord
import requests
from discord.ext import commands
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastHelperFunctions as fhf
import importlib
import time
import musicbrainzngs
import lyricsgenius as lg 


importlib.reload(fhf)
load_dotenv('3510.env')

TOKEN = os.getenv('DISCORD_TOKEN')
geniusClient = os.getenv("GENIUS_CLIENT_ID")
geniusSecret = os.getenv("GENIUS_CLIENT_SECRET")
geniusClientAccess = os.getenv("GENIUS_CLIENT_ACCESS_TOKEN")
API_key	=os.getenv('API_key')
shared_secret=os.getenv('shared_secret')

intents = discord.Intents().all()
nest_asyncio.apply()

#https://docs.python.org/3/library/asyncio.html
#https://discordpy.readthedocs.io/en/latest/api.html
#https://discordpy.readthedocs.io/en/latest/index.html
#https://discordpy.readthedocs.io/en/latest/intents.html

In [2]:
load_dotenv('3510.env')

True

In [3]:
bot = commands.Bot(command_prefix="/",intents = intents)

@bot.event 
async def on_ready():
    print("{1} {0} has connected to Discord.".format(bot.user.id,bot.user.name))

@bot.command(breif='hi bot',description='saying hello~!')
async def sayhello(ctx, *args):
    if (len(args)!= 0):
        for name in args:
            await ctx.send(f"Hello {name}")
    else:
        await ctx.send("""Hi There! I am a basic robot who can greet you! Whoop Whoop Whoop!""")
        await channel.send(f"On this server, I am trying to build a music reccomendation system!")

@bot.event
async def on_member_join(member):
    print(f'{member} has joined a server.')
    channel = discord.utils.get(bot.get_all_channels(),name='general')
    if channel: # figured out to use ''' to maintain spacing and make the ascii art appear properly.
        await channel.send("""```
 __      __ ___  _     ___  ___   __  __  ___ 
 \ \    / /| __|| |   / __|/ _ \ |  \/  || __|
  \ \/\/ / | _| | |__| (__| (_) || |\/| || _| 
   \_/\_/  |___||____|\___|\___/ |_|  |_||___|
                                              
        ```""")
        await channel.send(f"Hello {member.mention}!")
        await channel.send(f"Welcome to the server!")
        await channel.send(f"I am a recomendation bot!")
        await channel.send(f"Type  '/getRecs fav artist' to get similar artists!")
        await channel.send(f"Type  '/getRecsByCountry fav country' to get similar artists!")
        await channel.send(f"Type  '/plotSingers fav artist' to see similar artists!")
        await channel.send(f"Type  '/getLyrics song, artist' to see lyrics, thanks lyrics genius!")
        await channel.send(f"Type  '/getArtistId artist' to see a list of different artists you may mean!")

# Send cute AI photo
        with open('image.jpg', 'rb') as f:
            picture = discord.File(f)
            await channel.send(file=picture)

@bot.command(breif='Get Artist Recommendations',
            description='Get Similar Artist Recommendations by typing "/getRecs artist"~ Try it out!')
async def getRecs(ctx,*args):
    itemQueryString = "".join(args)
    recsListings = fhf.getArtistReccomendations(itemQueryString)
    print(len(recsListings))
    await ctx.send(recsListings)
    #recsDF = pd.DataFrame(recsListings,columns = ['Name','URL','Match'])
    for piece in recsListings:
        await ctx.send(piece)
    dataTableStringPieces = fhf.discordMessageSlicer(recsListings.to_string(),1000)
    for piece in dataTableStringPieces:
        await ctx.send(piece)
        await ctx.send("How are these Recs?")


@bot.command(breif='Get Top artists in foreign countries!',
            description='Get Artists Recommendations in foreign countries by typing "/getRecsByCountry artist" ~ Try it out!')
async def getRecsByCountry(ctx,*args):
    itemQueryString = "".join(args)
    recsListings = fhf.getArtistReccomendations(itemQueryString)
    print(len(recsListings))
    await ctx.send(recsListings)
    print(type(recsListings))
    #recsDF = pd.DataFrame(recsListings,columns = ['Name','URL','Match'])
    for piece in recsListings:
        await ctx.send(piece)
    dataTableStringPieces = fhf.discordMessageSlicer(recsListings.to_string(),1000)
    for piece in dataTableStringPieces:
        await ctx.send(piece)
        await ctx.send("How are these Recs?")   

# Lyrics Genius # Second Data Source |||| This didn't work ｜｜｜ After hours it now works
@bot.command(breif='Search up song lyrics',
            description='Found a new artist? Lets learn some songs of theirs!')
async def getLyrics(ctx,song,*,artist):
    lyrics = fhf.getSongLyrics(song,artist)
    if not lyrics:
        await ctx.send("Sorry, I couldn't find any lyrics for this song.")
    #time.sleep(5) # I thought if we gave it a second it would at least output the url but that is not relevant.
    else:
        #await ctx.send(lyrics)
        dataTableStringPieces = fhf.discordMessageSlicer(lyrics,1500)
        for piece in dataTableStringPieces:
            await ctx.send(piece)
            #await ctx.send('chunking...')#
    # await ctx.send(lyrics)



# Third Data Source Music Brainz
@bot.command(brief="Search up artist ID")
async def getArtistId(ctx, *args):
    itemQueryString = " ".join(args)
    results = fhf.findArtist(itemQueryString)
    await ctx.send(results)
    await ctx.send("Now, That you have these IDs, select the id you want to search with and copy and paste for the next function")
    await ctx.send("/getArtistInfo id ")
   

@bot.command(brief="Search up artist with the ID")
async def getArtistInfo(ctx, *args):
    itemQueryString = " ".join(args)
    results = fhf.getArtistInfo(itemQueryString,limit=5)
    if not results:
        await ctx.send(type(results))
        return f"No artist found with ID: {itemQueryString}"
    else:    
        await ctx.send(results)
        # dataTableStringPieces = fhf.discordMessageSlicer(results.to_string(),1500)
        # for piece in dataTableStringPieces:
        #     print(piece)
        #     await ctx.send(piece)
 


# Plot those artists!!
@bot.command(breif='Visualize artists matches', description='This command visualizes listed artists to provide a better viewing experience!')
async def plotSingers(ctx, *args):
    itemQueryString = " ".join(args)
    recsListings = fhf.getArtistReccomendations(itemQueryString)
    # restructure the list by splitting it by commas into chunks of 3 (one for each element in the list)
    # chunks = [recsListings[i:i+3] for i in range(0, len(recsListings), 3)] # list comprehension for the win... |||| Later I
    # realized that I should be doing this type of data clean up in as I make the requsts so this is commented out now
    recsDF = pd.DataFrame(recsListings, columns=['Name', 'URL', 'Match']) #making the df not just one big list [27:1] wasn't working for me
    recsDF['Match'] = pd.to_numeric(recsDF['Match'])
    recsDF = recsDF.sort_values(by='Match', ascending=False)
    plt.figure(figsize=(12, 6)) 
    artistBarplot = sns.barplot(x='Name', y='Match', data=recsDF,hue='Name', palette='viridis',legend=False)
    for index, row in recsDF.iterrows():
        plt.text(index, row['Match'], f"{row['Match']:.2f}", ha='center', va='bottom')
    plt.title('Similar Artists and Match Scores', fontsize=16)
    plt.xlabel('Artist Name', fontsize=12)
    plt.ylabel('Match Score', fontsize=12)
    plt.xticks(rotation=45, horizontalalignment='right', fontsize=10)
    plt.tight_layout()
    plt.savefig('singersbloxplot.png')
    plt.clf()
    fileObj = discord.File('singersbloxplot.png', filename='singersbloxplot.png')
    await ctx.send("Here are the similar artists:", file=fileObj)

bot.run(TOKEN)

[2024-12-13 19:42:43] [INFO    ] discord.client: logging in using static token
[2024-12-13 19:42:44] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 7bd41ff5ac4fb51fffe768159f3dab18).


music_recs 1313975035165278219 has connected to Discord.
